In [4]:
import os
import pandas as pd
import json
import pickle
import zipfile
import shutil

In [2]:
!pip install avro-python3
!pip install tf_slim==1.1.0
!pip install tf-models-official==2.7.0
!pip install lvis
!pip install tensorflow_io==0.23.1
!pip install keras==2.7.0
!pip install opencv-python-headless==4.5.2.52

  Preparing metadata (setup.py) ... done
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43993 sha256=66eaac669d4f38321a553dc1912ae9f9cd0b800a6c51f01c6ece4f6b1034a6c3
  Stored in directory: /root/.cache/pip/wheels/31/be/50/1145d9510eb4440893fc0ec676ef9464a05e0f7492a76fbb2c
Successfully built avro-python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB

In [3]:
!pip install tensorflow-object-detection-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.0/739.0 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection

In [5]:
labels = [{'name':'carro', 'id': 1}, {'name': 'motos', 'id': 2}]
with open("label_map.pbtxt", "w") as f:
  for label in labels:
    f.write('item { \n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

In [4]:
#Descargar los modelos pre-entrenados en este caso SSD + MobileNetV2
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz \
-O /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2025-05-23 22:31:21--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.15.251, 172.217.164.27, 172.217.0.91, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.15.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

/content/ssd_mobile 100%[===================>]  19.56M  --.-KB/s    in 0.06s   

2025-05-23 22:31:21 (304 MB/s) - ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [14]:
# descomprimir el archivo .gz
#!tar -zxvf /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
output_path = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almaceno en {}".format(output_path))

La carpeta se almaceno en /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8


In [7]:
# crear carpeta de entrenamineto
path_training = '/content/ssd_mobilenet'
os.mkdir(path_training)

In [8]:
# mover el archivo pipeline.config de la carpeta ssd_mobilenet_v2.... hacia la carpeta ssd_mobilenet
shutil.move('/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', path_training)

'/content/ssd_mobilenet/pipeline.config'

In [2]:
import tensorflow as tf
from google.protobuf import text_format

In [9]:
# crear el archivo de configuracio para que lo podamos editar
from object_detection.protos import pipeline_pb2
pipeline = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(os.path.join(path_training, 'pipeline.config'), 'r') as f:
  proto_str = f.read()
  # Remove the problematic field before parsing
  proto_str = proto_str.replace('fine_tune_checkpoint_version: V2', '')
  text_format.Merge(proto_str, pipeline)

In [11]:

label_map_path = '/content/label_map.pbtxt'
train_record_path = '/content/train.record'
test_record_path = '/content/test.record'

In [16]:
# Editar el archivo config de la arquitectura de mobilenet_v2

# Establecer el numero de clases
pipeline.model.ssd.num_classes = 2

# Establecer la cantidad del lote
pipeline.train_config.batch_size = 4

# Definimos desde donde quiero que empieze a entrenar , desde que checkpoint
pipeline.train_config.fine_tune_checkpoint = os.path.join(output_path, 'checkpoint/ckpt-0')

# Definir que tipo de entrenamiento queremos hacer
pipeline.train_config.fine_tune_checkpoint_type = 'detection'


#Obtener el path del label_map de train
pipeline.train_input_reader.label_map_path = label_map_path

#Leer el archivo tf record train
pipeline.train_input_reader.tf_record_input_reader.input_path[0] = train_record_path

# Obtener el path del label_map de test
pipeline.eval_input_reader[0].label_map_path = label_map_path

#Leer el archivo tf record test
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_path










In [13]:
# Downgrade protobuf to a compatible version
!pip uninstall protobuf -y
!pip install protobuf==3.20.x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
